In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

bs, expansion, dim = 2, 4, 3
X = torch.randn(bs, dim)

H_pre = torch.randn(expansion)
X_pre = torch.einsum('bed,e->bd', X.unsqueeze(1).repeat(1, expansion, 1), H_pre)
X_pre

tensor([[ 1.3193,  0.2545,  0.4482],
        [-0.1236,  0.4518,  0.5052]])

In [6]:
class HC(nn.Module):
    def __init__(self, n, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n = n
        self.alpha = nn.Parameter(torch.randn(n))
        self.beta = nn.Parameter(torch.randn(n))
        self.interaction = nn.Parameter(torch.randn(n, n))

    def forward(self, H, layer):
        res = torch.einsum('bled,ee->bled', H, self.interaction)
        H = torch.einsum('bled,e->bld', H, self.alpha)
        H = layer(H)
        H = torch.einsum('bld,e->bled', H, self.beta)
        return H + res

hc = HC(expansion)
hc(torch.randn(bs, sl, dim).unsqueeze(-2).repeat(1, 1, expansion, 1), nn.Identity()).shape

tensor([[[-1.1936, -0.2303, -0.4055],
         [-1.1936, -0.2303, -0.4055],
         [-1.1936, -0.2303, -0.4055],
         [-1.1936, -0.2303, -0.4055]],

        [[ 0.1118, -0.4088, -0.4571],
         [ 0.1118, -0.4088, -0.4571],
         [ 0.1118, -0.4088, -0.4571],
         [ 0.1118, -0.4088, -0.4571]]])

In [8]:
class HC(nn.Module):
    def __init__(self, n, d, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n = n
        self.d = d

        self.alpha = nn.Parameter(torch.randn(n))
        self.beta = nn.Parameter(torch.randn(n))
        self.interaction = nn.Parameter(torch.randn(n, n))

    def forward(self, H, layer):
        res = torch.einsum('bed,ee->bed', H, self.interaction)
        H = torch.einsum('bed,e->bd', H, self.alpha)
        H = layer(H)
        H = torch.einsum('bd,e->bed', H, self.beta)
        return H + res

hc = HC(expansion, dim)
hc(torch.randn(bs, expansion, dim), nn.Identity())

tensor([[[ 0.2969,  0.5292, -1.1873],
         [ 3.6162, -1.0307, -5.7725],
         [ 0.2845, -0.2101, -0.9452],
         [-2.1065,  0.7294,  4.9829]],

        [[ 1.2141,  0.1998,  1.6406],
         [ 5.8315,  7.1558, 10.5457],
         [ 0.5200,  0.6758,  1.4293],
         [-4.8077, -5.2147, -8.0921]]], grad_fn=<AddBackward0>)